2025-10-19 23:31:38 Sunday

论文阅读笔记：Stabilizing MoE Reinforcement Learning by Aligning Training and Inference Routers

## 📄 基本信息

* **标题**：Stabilizing MoE Reinforcement Learning by Aligning Training and Inference Routers
* **作者**：Wenhan Ma, Hailin Zhang, Liang Zhao, Yifan Song, Yudong Wang, Zhifang Sui, Fuli Luo
* **单位**：Peking University & Xiaomi LLM-Core
* **年份**：2025
* **论文链接**：[arXiv:2510.11370v1](https://arxiv.org/abs/2510.11370v1)
* **模型参考**：Qwen3-30B-A3B（MoE架构）

## 🎯 一句话总结

> 提出 **Rollout Routing Replay (R3)** 方法，将 MoE 推理阶段的路由分布（routing mask）在训练中重放（replay），从而对齐训练和推理阶段的专家选择行为，显著稳定了 RL 训练并避免 MoE RL 的坍塌。

---

## 🧩 1. 研究背景与问题定义

### 研究目标

* 解决 **Mixture-of-Experts (MoE)** 模型在强化学习阶段的不稳定问题。
* 发现：MoE 中的路由机制（router）在训练与推理阶段表现不一致，是导致崩溃的核心原因。

### 现有问题

* MoE 在grpo/ppo训练中不稳定，这是由于两个原因导致的，从损失函数可见
$$
J(\theta) = \mathbb{E}_{x,y \sim \pi_{\text{old}}} \left[ \frac{1}{|y|} \sum_{t} \min \left( w_t(\theta) A_t, \text{clip}(w_t(\theta), 1 - \epsilon, 1 + \epsilon) A_t \right) \right]
$$
其中：$w_t(\theta) = \frac{\pi_{\theta}(y_t | x, y_{<t})}{\pi_{\text{old}}(y_t | x, y_{<t})}$

此时$\pi_{\text{old}}$来自于推理引擎（SGLang），$\pi_{\theta}$来自于训练引擎（Megatron），但由于框架差异与算子实现不同，它们在相同输入下输出的 token 概率分布并不完全一致。
并且由于moe的机制使得不同输入的专家选择不同，导致$\pi_{\theta}$和$\pi_{old}$的分布差异度要大于dense模型；
这样都使得了$w_t(\theta)$会过大和过小，从而训练不稳定。


### 作者动机

- 以往方法（如 TIS, GSPO）通过修正 importance ratio 或修改优化目标来缓解 off-policy 问题，
但它们都未触及 MoE 路由机制导致的分布不一致这一根本原因。

- 作者提出 Rollout Routing Replay (R3)，从根本上对齐推理与训练阶段的路由分布，
使得两个阶段在专家选择上保持一致，从而消除 MoE 引起的额外偏差。


---

## ⚙️ 2. 方法概述（核心创新）

### 核心思想：Rollout Routing Replay (R3)

* **思路**：
  在强化学习优化中计算重要性比值
  $$
  w_t(\theta) = \frac{\pi_{\theta}(y_t|x, y_{<t})}{\pi_{\text{old}}(y_t|x, y_{<t})},
  $$
  时，R3 让**新策略 $\pi_\theta$ 在计算分子时，使用旧策略（推理阶段）所选择的专家集合（Top-K mask）**。
  这样可使训练和推理阶段的路由分布一致，从而减小分布漂移。

* **方法代价**：
  每轮训练都需要一次新的推理来生成路由 mask，供下一次优化使用。
  因此，虽然不增加额外的推理次数，但 rollout 阶段的时间略有增加（约 3–5%）。

* **效果**：
  显著降低训练–推理 KL 散度，极端 token 比例（训练概率/推理概率比 > 2）从约 10% 降至 0.01%。


### 具体过程

#### 常规 MoE 前向（训练引擎内部）

路由器正常执行如下步骤：

$$
s^{\text{train}} = x^{\text{train}} W_r
$$

$$
I^{\text{train}} = \text{TopKMask}(s^{\text{train}}, K)
$$

$$
g^{\text{train}}_i = \frac{I^{\text{train}}_i e^{s^{\text{train}}_i}}{\sum_j I^{\text{train}}_j e^{s^{\text{train}}_j}}
$$

$$
y^{\text{train}} = \sum_i g^{\text{train}}_i , E_i(x^{\text{train}})
$$


#### R3 的关键改动：**只重放 mask，不重放 logits/权重**

在 R3 中，推理阶段得到：

$$
s^{\text{infer}} = x^{\text{infer}} W_r, \quad
I^{\text{infer}} = \text{TopKMask}(s^{\text{infer}}, K)
$$

训练阶段重放推理时的专家选择：

$$
g^{\text{replay}}_i
= \frac{I^{\text{infer}}_i e^{s^{\text{train}}_i}}
{\sum_j I^{\text{infer}}_j e^{s^{\text{train}}_j}},
\qquad
y^{\text{replay}} = \sum_i g^{\text{replay}}_i , E_i(x^{\text{train}})
$$


### **好处**

1. **专家集合一致**：训练与推理阶段使用相同的专家集合，消除离散 Top-K 选择造成的不连续性。
2. **保留梯度**：仍对 $s^{\text{train}}$ 可微，可同时更新 router 与 experts 参数。
3. **计算路径一致**：不改变算量路径，几乎无额外计算开销。





## 🔬 3. 实验与结果分析

### 📊 数据与任务

* 任务：数学推理 RL（Math Reasoning）
* 数据集：BigMath、ORZ 等 → 10 万个可验证数学问题
* 验证集：AIME24 / AIME25 / AMC23 / MATH500 Lv5

### 🧪 对比方法

* GRPO（基线）
* GSPO（Group Sequence Policy Optimization）
* TIS（Truncated Importance Sampling）
* R3 可与它们结合（R3+TIS, R3+GSPO）

### 📈 实验1：方法有效性对比

![结果](pic/Snipaste_2025-10-20_00-38-31.png)



➡ R3 平均提升 **+5~10 分**，完全避免训练崩溃。


### 实验2: 训练动态分析

![训练动态分析](pic/Snipaste_2025-10-20_00-41-09.png)


#### 🧩 **图 5：训练–推理一致性分析（Training–Inference Collapse Analysis）**

**内容概述：**
图 5 展示了在 MoE 模型强化学习中，随训练步数变化的两项关键指标：

1. **Training–Inference KL Divergence**

   * 衡量训练策略 $\pi_{\text{train}}$ 与推理策略 $\pi_{\text{infer}}$ 的分布差异。
   * 未使用 R3 时，KL 在训练中持续上升，表示分布逐步漂移；
   * 加入 **R3** 后，KL 明显降低并保持稳定，说明训练与推理分布对齐良好。

2. **Extreme Token Fraction $F(\tau=2)$**

   * 衡量训练与推理概率比值超过 2 的 token 比例。
   * 无 R3 时，极端 token 比例迅速增加，训练易崩溃；
   * 有 R3 后，比例下降约两个数量级（约 10% → 0.01%），模型输出分布更加一致。

**结论：**

> 图 5 证明了 R3 可以有效降低训练–推理分布差异，消除 MoE 模型在 RL 训练中的 “collapse” 现象，使重要性采样比值 $w_t$ 稳定、训练可持续。

---

#### 🧩 **图 6：训练动态分析（Training Dynamics of Qwen3-30B-A3B-Base）**

**内容概述：**
图 6 展示了模型在不同训练配置下（是否启用 TIS 与 R3）随训练步数变化的整体训练动态指标：

* **Response Length**：无 R3 时响应长度快速增长、训练发散；R3 使长度稳定。
* **Gradient Norm**：无 R3 时梯度范数震荡剧烈；R3 显著平滑曲线，防止梯度爆炸。
* **Entropy**：R3 维持较高熵值，模型保持探索性；无 R3 时熵迅速塌陷。
* **Average Validation Score**：使用 R3 后验证得分稳步上升并达到更高峰值。

**结论：**

> 图 6 说明 R3 能显著稳定 MoE 强化学习过程，使梯度、熵、响应长度等指标保持健康分布，从而带来更好的收敛与泛化性能。





---

## 💡 4. 主要结论与启示

* MoE 模型在 RL 中的不稳定**主要来自 routing 不一致**，而非采样或核级随机性。
* R3 提供一种低成本的对齐方式，使 MoE RL 稳定性接近 Dense 模型。
* 对未来方向：

  * R3 可与 router regularization / expert balance loss 结合；
  * 可探索自适应 mask replay（部分随机替换）。

---